<a href="https://colab.research.google.com/github/Akshitkt001/Music_GenAI/blob/main/LLM_Voice_Changer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install librosa

In [ ]:
!pip install numpy

In [ ]:
!pip install soundfile


In [ ]:
!pip install torchvision

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install scipy

In [ ]:
!pip install torch

In [ ]:
import librosa
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Load the source and target singer's voice samples
source_singer, _ = librosa.load('test1.wav', sr=None)
target_singer, _ = librosa.load('test2.wav', sr=None)

# Extract audio features (e.g., Mel-frequency cepstral coefficients - MFCCs)
def extract_features(audio):
    mfccs = librosa.feature.mfcc(y=audio, sr=44100)  # Adjust sample rate if needed
    return mfccs

source_features = extract_features(source_singer)
target_features = extract_features(target_singer)

# Define a simple voice conversion model
class VoiceConverter(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(VoiceConverter, self).__init__()
        self.fc1 = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc1(x)

# Determine input and output dimensions
input_dim = source_features.shape[0] * source_features.shape[1]
output_dim = target_features.shape[0] * target_features.shape[1]

# Initialize the model and optimizer
model = VoiceConverter(input_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Flatten the features for training
source_features_flat = source_features.reshape(-1, input_dim)
target_features_flat = target_features.reshape(-1, output_dim)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    converted_features_flat = model(torch.Tensor(source_features_flat))

    # Reshape the converted features to match the target shape
    converted_features = converted_features_flat.view(target_features.shape)

    # Define a loss function (you might need more advanced loss functions)
    loss = nn.MSELoss()(converted_features, torch.Tensor(target_features))

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Generate converted audio using the trained model
converted_mfccs = model(torch.Tensor(source_features_flat)).detach().numpy()
converted_mfccs = converted_mfccs.reshape(target_features.shape)
converted_audio = librosa.feature.inverse.mfcc_to_audio(converted_mfccs)

In [ ]:
# Save the converted audio
import soundfile as sf

sf.write('converted_song.wav', converted_audio, 44100)